In [6]:
# adapted from https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
# https://www.youtube.com/watch?v=MqugtGD605k
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, RepeatVector, Dense
import numpy as np
import wandb
from wandb.keras import WandbCallback

wandb.init()
config = wandb.config

class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

# Parameters for the model and dataset.
config.hidden_size = 128
config.batch_size = 128

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
maxlen = 30
output_len = 10
# All the numbers, plus sign and space for padding.
chars = '0123456789+$-,/.ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuwxyv '
ctable = CharacterTable(chars)


import pandas as pd
df = pd.read_csv("sale_prices.csv")
df = df.fillna(0)



raw_price = df["Price on site"]
parced_price = []

int_min_price = [int(price) for price in df["Min price"]]
int_max_price = [int(price) for price in df["Max price"]]
for idx in range(len(int_min_price)):
    toappend = str(int_min_price[idx])
    toappend+=' '*(output_len - len(str(int_min_price[idx])))
#     toappend+=' '*(30 - len(str(int_min_price[idx])) - len(str(int_max_price[idx])))
#     toappend+=str(int_max_price[idx])
    parced_price.append(toappend)

print('Total prices:', len(raw_price))

print('Vectorization...')
x = np.zeros((len(raw_price), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(raw_price), output_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(raw_price):
    x[i] = ctable.encode(str(sentence), maxlen)
for i, sentence in enumerate(parced_price):
    y[i] = ctable.encode(str(sentence), output_len)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

model = Sequential()
model.add(LSTM(config.hidden_size, input_shape=(maxlen, len(chars))))
model.add(RepeatVector(output_len))
model.add(LSTM(config.hidden_size, return_sequences=True))
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
             metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=config.batch_size,
              epochs=1,
              validation_data=(x_val, y_val),callbacks=[WandbCallback()])
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print('☑', end=' ')
        else:
            print('☒', end=' ')
        print('G',guess, end='\n')


Total prices: 638
Vectorization...
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 128)               100864    
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 10, 128)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 10, 128)           131584    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 10, 68)            8772      
Total params: 241,220
Trainable params: 241,220
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 1
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 4s 7ms/sample - loss: 4.2028 - acc: 0.3087 - val_loss: 4.15

575/575 [==============================] - 1s 2ms/sample - loss: 1.3489 - acc: 0.5532 - val_loss: 1.2767 - val_acc: 0.6175
Q $189,900                       T 189900     ☒ G 000000    
Q $1,295,000                     T 1295000    ☒ G 000000    
Q $64,900                        T 64900      ☒ G 000000    
Q $850,000                       T 850000     ☒ G 000000    
Q $1,149,900+                    T 1149900    ☒ G 000000    
Q $682,000                       T 682000     ☒ G 000000    
Q $599,900                       T 599900     ☒ G 000000    
Q $1,199,500                     T 1199500    ☒ G 000000    
Q $465,000                       T 465000     ☒ G 000000    
Q $414,000                       T 414000     ☒ G 000000    

--------------------------------------------------
Iteration 11
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 1.2702 - acc: 0.6110 - val_loss: 1.2148 - val_acc: 0.5587
Q $735,000                       T

575/575 [==============================] - 1s 2ms/sample - loss: 1.0823 - acc: 0.6466 - val_loss: 1.0312 - val_acc: 0.6476
Q $1,800,000                     T 1800000    ☒ G 190000    
Q $648,000                       T 648000     ☒ G 190000    
Q $4,875,000                     T 4875000    ☒ G 190000    
Q $98,000                        T 98000      ☒ G 190000    
Q $625,000                       T 625000     ☒ G 190000    
Q 0                              T 0          ☒ G 190000    
Q $3,650,000                     T 3650000    ☒ G 190000    
Q $89,000                        T 89000      ☒ G 190000    
Q $3,650,000                     T 3650000    ☒ G 190000    
Q $456,000 - $9,062,000          T 456000     ☒ G 190000    

--------------------------------------------------
Iteration 21
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 1.0752 - acc: 0.6466 - val_loss: 1.0262 - val_acc: 0.6556
Q $279,000                       T

575/575 [==============================] - 1s 2ms/sample - loss: 1.0653 - acc: 0.6442 - val_loss: 1.0261 - val_acc: 0.6476
Q $87,000                        T 87000      ☒ G 190000    
Q $625,000                       T 625000     ☒ G 190000    
Q $648,000                       T 648000     ☒ G 190000    
Q $489,500                       T 489500     ☒ G 190000    
Q $735,000                       T 735000     ☒ G 190000    
Q $489,500                       T 489500     ☒ G 190000    
Q $749,500                       T 749500     ☒ G 190000    
Q $77,500                        T 77500      ☒ G 190000    
Q $1,800,000                     T 1800000    ☒ G 190000    
Q $1,450,000                     T 1450000    ☒ G 190000    

--------------------------------------------------
Iteration 31
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 1.0780 - acc: 0.6454 - val_loss: 1.0250 - val_acc: 0.6302
Q $77,500                        T

575/575 [==============================] - 1s 2ms/sample - loss: 1.0574 - acc: 0.6520 - val_loss: 1.0120 - val_acc: 0.6556
Q $456,000 - $9,062,000          T 456000     ☒ G 199000    
Q $1,645,000                     T 1645000    ☒ G 199000    
Q $648,000                       T 648000     ☒ G 199000    
Q $4,875,000                     T 4875000    ☒ G 199000    
Q $1,350,000                     T 1350000    ☒ G 199000    
Q $3,118,000                     T 3118000    ☒ G 199000    
Q $189,900                       T 189900     ☒ G 199000    
Q $899,000                       T 899000     ☒ G 199000    
Q $995,000                       T 995000     ☒ G 199000    
Q $1,250,000                     T 1250000    ☒ G 199000    

--------------------------------------------------
Iteration 41
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 1.0552 - acc: 0.6520 - val_loss: 1.0106 - val_acc: 0.6556
Q $1,800,000                     T

...

575/575 [==============================] - 1s 2ms/sample - loss: 0.8291 - acc: 0.7259 - val_loss: 0.7906 - val_acc: 0.7365
Q $250,000                       T 250000     ☒ G 199000    
Q $1,350,000                     T 1350000    ☒ G 1190000   
Q $180,000                       T 180000     ☒ G 199000    
Q $3,118,000                     T 3118000    ☒ G 1190000   
Q $98,000                        T 98000      ☒ G 19000     
Q $519,000                       T 519000     ☒ G 199000    
Q $500,000                       T 500000     ☒ G 19000     
Q $625,000                       T 625000     ☒ G 199000    
Q $500,000                       T 500000     ☒ G 19000     
Q $682,000                       T 682000     ☒ G 199000    

--------------------------------------------------
Iteration 151
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 0.8279 - acc: 0.7264 - val_loss: 0.7870 - val_acc: 0.7381
Q $950,000                       

575/575 [==============================] - 1s 2ms/sample - loss: 0.8726 - acc: 0.6977 - val_loss: 0.8176 - val_acc: 0.7159
Q $519,000                       T 519000     ☒ G 199000    
Q $1,699,999                     T 1699999    ☒ G 1190000   
Q $189,900                       T 189900     ☒ G 199000    
Q $1,350,000                     T 1350000    ☒ G 1190000   
Q $950,000                       T 950000     ☒ G 19000     
Q $189,900                       T 189900     ☒ G 199000    
Q $465,000                       T 465000     ☒ G 190000    
Q $1,199,500                     T 1199500    ☒ G 1190000   
Q $749,000                       T 749000     ☒ G 190000    
Q $1,022,250 - $2,800,000        T 1022250    ☒ G 1190000   

--------------------------------------------------
Iteration 161
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 0.8838 - acc: 0.6950 - val_loss: 0.8643 - val_acc: 0.6746
Q $279,000                       

575/575 [==============================] - 1s 2ms/sample - loss: 0.8147 - acc: 0.7294 - val_loss: 0.7775 - val_acc: 0.7365
Q $5,200,000                     T 5200000    ☒ G 1190000   
Q $62,500                        T 62500      ☒ G 29000     
Q $1,199,500                     T 1199500    ☒ G 1190000   
Q $1,295,000                     T 1295000    ☒ G 1190000   
Q $850,000                       T 850000     ☒ G 290000    
Q -                              T 0          ☑ G 0         
Q $847,500                       T 847500     ☒ G 199000    
Q $950,000                       T 950000     ☒ G 290000    
Q $749,500                       T 749500     ☒ G 199000    
Q $1,250,000                     T 1250000    ☒ G 1190000   

--------------------------------------------------
Iteration 171
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 0.8122 - acc: 0.7287 - val_loss: 0.7760 - val_acc: 0.7381
Q $87,000                        

575/575 [==============================] - 1s 2ms/sample - loss: 0.8143 - acc: 0.7224 - val_loss: 0.7699 - val_acc: 0.7397
Q -                              T 0          ☑ G 0         
Q $8,998,000                     T 8998000    ☒ G 1190000   
Q $1,022,250 - $2,800,000        T 1022250    ☒ G 1190000   
Q $519,000                       T 519000     ☒ G 199000    
Q $279,000                       T 279000     ☒ G 199000    
Q $1,150,000                     T 1150000    ☒ G 1190000   
Q $625,000                       T 625000     ☒ G 199000    
Q $64,900                        T 64900      ☒ G 29000     
Q $500,000                       T 500000     ☒ G 290000    
Q $1,800,000                     T 1800000    ☒ G 1990000   

--------------------------------------------------
Iteration 181
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 0.8041 - acc: 0.7308 - val_loss: 0.7707 - val_acc: 0.7413
Q $89,000                        

575/575 [==============================] - 1s 2ms/sample - loss: 0.7793 - acc: 0.7360 - val_loss: 0.7509 - val_acc: 0.7397
Q $1,350,000                     T 1350000    ☒ G 1190000   
Q $250,000                       T 250000     ☒ G 290000    
Q $98,000                        T 98000      ☒ G 59000     
Q $87,000                        T 87000      ☒ G 59000     
Q 0                              T 0          ☑ G 0         
Q $4,875,000                     T 4875000    ☒ G 1990000   
Q $1,022,250 - $2,800,000        T 1022250    ☒ G 1190000   
Q $64,900                        T 64900      ☒ G 29000     
Q $1,699,999                     T 1699999    ☒ G 1190000   
Q $5,200,000                     T 5200000    ☒ G 1990000   

--------------------------------------------------
Iteration 191
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 0.7714 - acc: 0.7369 - val_loss: 0.7470 - val_acc: 0.7365
Q $1,199,500                     

In [7]:
for iteration in range(1, 500):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=config.batch_size,
              epochs=1,
              validation_data=(x_val, y_val),callbacks=[WandbCallback()])
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print('☑', end=' ')
        else:
            print('☒', end=' ')
        print('G',guess, end='\n')


--------------------------------------------------
Iteration 1
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 2s 4ms/sample - loss: 0.6581 - acc: 0.7817 - val_loss: 0.6426 - val_acc: 0.7952
Q $1,199,500                     T 1199500    ☒ G 1199900   
Q $1,750,000                     T 1750000    ☑ G 1750000   
Q $749,500                       T 749500     ☒ G 199000    
Q $279,000                       T 279000     ☒ G 195000    
Q $850,000                       T 850000     ☒ G 200000    
Q $1,300,000                     T 1300000    ☒ G 1500000   
Q $682,000                       T 682000     ☒ G 150000    
Q $1,022,250 - $2,800,000        T 1022250    ☒ G 1195000   
Q $3,650,000                     T 3650000    ☒ G 1750000   
Q $1,295,000                     T 1295000    ☒ G 1195000   

--------------------------------------------------
Iteration 2
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms

575/575 [==============================] - 1s 2ms/sample - loss: 0.6186 - acc: 0.7943 - val_loss: 0.6074 - val_acc: 0.7984
Q $995,000                       T 995000     ☒ G 299000    
Q $899,000                       T 899000     ☒ G 999000    
Q $62,500                        T 62500      ☒ G 19500     
Q $3,118,000                     T 3118000    ☒ G 1175000   
Q $1,699,999                     T 1699999    ☒ G 1999999   
Q $1,750,000                     T 1750000    ☒ G 1150000   
Q $3,650,000                     T 3650000    ☒ G 1150000   
Q $279,000                       T 279000     ☒ G 199000    
Q $414,000                       T 414000     ☒ G 199000    
Q $13,500,000                    T 13500000   ☒ G 1150000   

--------------------------------------------------
Iteration 12
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 0.5898 - acc: 0.8043 - val_loss: 0.5969 - val_acc: 0.8159
Q $1,645,000                     T

575/575 [==============================] - 1s 2ms/sample - loss: 0.5443 - acc: 0.8256 - val_loss: 0.5603 - val_acc: 0.8175
Q $1,450,000                     T 1450000    ☒ G 1250000   
Q -                              T 0          ☑ G 0         
Q $79,900                        T 79900      ☒ G 49900     
Q $1,645,000                     T 1645000    ☒ G 1155000   
Q $79,900                        T 79900      ☒ G 49900     
Q $77,500                        T 77500      ☒ G 17500     
Q $3,118,000                     T 3118000    ☒ G 1175000   
Q $87,000                        T 87000      ☒ G 25000     
Q $1,150,000                     T 1150000    ☑ G 1150000   
Q $1,250,000                     T 1250000    ☒ G 1150000   

--------------------------------------------------
Iteration 22
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 0.5411 - acc: 0.8226 - val_loss: 0.5573 - val_acc: 0.8111
Q $2,750,000                     T

575/575 [==============================] - 1s 2ms/sample - loss: 0.5237 - acc: 0.8280 - val_loss: 0.5479 - val_acc: 0.8270
Q -                              T 0          ☑ G 0         
Q $1,350,000                     T 1350000    ☒ G 1150000   
Q $339,900                       T 339900     ☒ G 199900    
Q $1,080,000                     T 1080000    ☒ G 1500000   
Q $749,500                       T 749500     ☒ G 149900    
Q $25,000                        T 25000      ☒ G 15000     
Q $599,900                       T 599900     ☒ G 999900    
Q $1,750,000                     T 1750000    ☒ G 1150000   
Q $5,200,000                     T 5200000    ☒ G 1500000   
Q $89,000                        T 89000      ☒ G 49000     

--------------------------------------------------
Iteration 32
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 0.5229 - acc: 0.8311 - val_loss: 0.5399 - val_acc: 0.8286
Q $749,000                       T

575/575 [==============================] - 1s 2ms/sample - loss: 0.5107 - acc: 0.8304 - val_loss: 0.5276 - val_acc: 0.8270
Q $599,900                       T 599900     ☒ G 999900    
Q $2,195,900                     T 2195900    ☒ G 1299000   
Q $2,750,000                     T 2750000    ☒ G 1250000   
Q $500,000                       T 500000     ☑ G 500000    
Q $13,500,000                    T 13500000   ☒ G 11500000  
Q $465,000                       T 465000     ☒ G 175000    
Q $1,250,000                     T 1250000    ☒ G 1150000   
Q -                              T 0          ☑ G 0         
Q $850,000                       T 850000     ☒ G 250000    
Q $87,000                        T 87000      ☒ G 25000     

--------------------------------------------------
Iteration 42
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 0.5092 - acc: 0.8297 - val_loss: 0.5272 - val_acc: 0.8317
Q $1,199,500                     T

...

575/575 [==============================] - 1s 1ms/sample - loss: 0.0628 - acc: 0.9847 - val_loss: 0.3716 - val_acc: 0.9048
Q $2,750,000                     T 2750000    ☑ G 2750000   
Q $500,000                       T 500000     ☑ G 500000    
Q $648,000                       T 648000     ☒ G 649000    
Q $648,000                       T 648000     ☒ G 649000    
Q $648,000                       T 648000     ☒ G 649000    
Q $250,000                       T 250000     ☑ G 250000    
Q $735,000                       T 735000     ☒ G 775000    
Q $648,000                       T 648000     ☒ G 649000    
Q $5,200,000                     T 5200000    ☑ G 5200000   
Q $180,000                       T 180000     ☑ G 180000    

--------------------------------------------------
Iteration 452
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 1ms/sample - loss: 0.1011 - acc: 0.9656 - val_loss: 0.4614 - val_acc: 0.8794
Q $456,000 - $9,062,000          

575/575 [==============================] - 1s 2ms/sample - loss: 0.1216 - acc: 0.9593 - val_loss: 0.4226 - val_acc: 0.8857
Q -                              T 0          ☑ G 0         
Q $1,149,900+                    T 1149900    ☑ G 1149900   
Q $975,000                       T 975000     ☒ G 775000    
Q $1,450,000                     T 1450000    ☑ G 1450000   
Q $1,295,000                     T 1295000    ☒ G 1229000   
Q $4,875,000                     T 4875000    ☒ G 287500    
Q $749,500                       T 749500     ☒ G 779000    
Q $749,500                       T 749500     ☒ G 779000    
Q $2,750,000                     T 2750000    ☑ G 2750000   
Q $1,450,000                     T 1450000    ☑ G 1450000   

--------------------------------------------------
Iteration 462
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 1ms/sample - loss: 0.0966 - acc: 0.9710 - val_loss: 0.3655 - val_acc: 0.8984
Q $850,000                       

575/575 [==============================] - 1s 2ms/sample - loss: 0.0321 - acc: 0.9965 - val_loss: 0.3604 - val_acc: 0.9000
Q $189,900                       T 189900     ☑ G 189900    
Q $3,118,000                     T 3118000    ☒ G 1133000   
Q $2,195,900                     T 2195900    ☒ G 1199900   
Q $899,000                       T 899000     ☑ G 899000    
Q $250,000                       T 250000     ☑ G 250000    
Q $625,000                       T 625000     ☑ G 625000    
Q $1,385,000                     T 1385000    ☑ G 1385000   
Q $210,000                       T 210000     ☒ G 220000    
Q $25,000                        T 25000      ☑ G 25000     
Q $1,800,000                     T 1800000    ☑ G 1800000   

--------------------------------------------------
Iteration 472
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 0.0296 - acc: 0.9979 - val_loss: 0.3326 - val_acc: 0.9063
Q $465,000                       

575/575 [==============================] - 1s 2ms/sample - loss: 0.0228 - acc: 0.9981 - val_loss: 0.3484 - val_acc: 0.9016
Q $1,750,000                     T 1750000    ☑ G 1750000   
Q 0                              T 0          ☑ G 0         
Q $735,000                       T 735000     ☒ G 375000    
Q 0                              T 0          ☑ G 0         
Q $3,585,000+                    T 3585000    ☒ G 3855000   
Q $2,750,000                     T 2750000    ☑ G 2750000   
Q $25,000                        T 25000      ☑ G 25000     
Q $904,500                       T 904500     ☒ G 549000    
Q $217,260                       T 217260     ☒ G 1277700   
Q $2,750,000                     T 2750000    ☑ G 2750000   

--------------------------------------------------
Iteration 482
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 0.0225 - acc: 0.9984 - val_loss: 0.3500 - val_acc: 0.9016
Q -                              

575/575 [==============================] - 1s 2ms/sample - loss: 0.0200 - acc: 0.9988 - val_loss: 0.3586 - val_acc: 0.9016
Q $1,199,500                     T 1199500    ☒ G 1199900   
Q $648,000                       T 648000     ☒ G 649000    
Q $465,000                       T 465000     ☑ G 465000    
Q $1,385,000                     T 1385000    ☑ G 1385000   
Q $749,500                       T 749500     ☒ G 749000    
Q $189,900                       T 189900     ☑ G 189900    
Q $465,000                       T 465000     ☑ G 465000    
Q $180,000                       T 180000     ☑ G 180000    
Q $1,022,250 - $2,800,000        T 1022250    ☑ G 1022250   
Q $25,000                        T 25000      ☑ G 25000     

--------------------------------------------------
Iteration 492
Train on 575 samples, validate on 63 samples
575/575 [==============================] - 1s 2ms/sample - loss: 0.0199 - acc: 0.9988 - val_loss: 0.3623 - val_acc: 0.9048
Q $3,585,000+                    